# 📄 生成 AT 指令文档（支持多类型 + 响应校正优先）

In [17]:
!pip install pandas jinja2 sphinx sphinx-rtd-theme

In [18]:
import os
import pandas as pd
import json
from jinja2 import Environment
from collections import defaultdict
import re

In [19]:
# 若没有 conf.py 则自动创建
conf_path = "docs/source/conf.py"
if not os.path.exists(conf_path):
    os.makedirs(os.path.dirname(conf_path), exist_ok=True)
    with open(conf_path, "w", encoding="utf-8") as f:
        f.write("""
import os
import sys
sys.path.insert(0, os.path.abspath('.'))

project = 'AT Command Manual'
author = 'Your Name'
release = '1.0'

extensions = []
templates_path = ['_templates']
exclude_patterns = []

html_theme = 'sphinx_rtd_theme'
html_static_path = ['_static']

def setup(app):
    app.add_css_file('custom.css')
""")
    print("✅ conf.py 已创建")
else:
    print("ℹ️ conf.py 已存在")

static_dir = 'docs/source/_static'
os.makedirs(static_dir, exist_ok=True)
with open(os.path.join(static_dir, 'custom.css'), 'w', encoding='utf-8') as f:
    f.write("body { font-family: 'Microsoft YaHei', sans-serif; }")
print("✅ CSS 准备完成")

ℹ️ conf.py 已存在
✅ CSS 准备完成


In [20]:
# 读取 CSV & 定义模板
CSV_PATH = 'at_commands_full_template.csv'  # 使用你新模板的 CSV 名称
OUTPUT_DIR = 'docs/source'

TEMPLATE_STRING = """
.. _cmd-{{ command_name|lower }}:

:ref:`{{ command_name }} <Basic-AT>`：{{ command_title }}
--------------------------------------------------------

{{ description or '' }}

执行命令
^^^^^^^^

**命令：**

::

    {{ command }}

**响应：**

::

{% for line in response.splitlines() %}    {{ line }}
{% endfor %}

参数
^^^^
{% if parameters %}
{% for name, data in parameters.items() %}
- **{{ name }}**：
  {% if data.get('__desc__') %}
    {{ data['__desc__'] }}
  {% endif %}
  {% for key, value in data.items() if key != '__desc__' %}
    - {{ key }}：{{ value }}
  {% endfor %}
{% endfor %}
{% else %}无
{% endif %}

说明
^^^^
{{ note or '无' }}

示例命令
^^^^^^^^

::

{% for line in example.splitlines() %}    {{ line }}
{% endfor %}
"""

In [21]:
def sanitize_filename(fn):
    return re.sub(r'[<>:"/\\|?*\r\n]', '', fn)

def expand_multitype_row(row):
    types = [t.strip() for t in str(row.get('命令类型', '')).split(';')]
    cmds = [t.strip() for t in str(row.get('命令格式', '')).split(';')]
    # 用响应校正优先字段，否则响应字段
    raw_resp_all = str(row.get('响应校正', '')).strip().strip("'''")
    if raw_resp_all:
        resps = raw_resp_all.split(';')
    else:
        resps = [t.strip() for t in str(row.get('响应', '')).split(';')]
    examples = [t.strip() for t in str(row.get('示例命令', '')).split(';')]

    max_len = max(len(types), len(cmds), len(resps), len(examples))
    expanded = []
    for i in range(max_len):
        expanded.append({
            '章节': row.get('章节'),
            '命令': row.get('命令'),
            '命令标题': row.get('命令标题'),
            '命令类型': types[i] if i < len(types) else '',
            '命令格式': cmds[i] if i < len(cmds) else '',
            '响应': resps[i] if i < len(resps) else '',
            '示例命令': examples[i] if i < len(examples) else '',
            '功能描述': row.get('功能描述'),
            '备注': row.get('备注'),
            '表数量': row.get('表数量'),
            '顺序': f"{row.get('顺序')}.{i+1}",
            '参数json': row.get('参数json')
        })
    return expanded

In [22]:
# 读取并展开多类型行
df0 = pd.read_csv(CSV_PATH, dtype=str).fillna("")
expanded = []
for _, row in df0.iterrows():
    expanded.extend(expand_multitype_row(row))
df = pd.DataFrame(expanded)
df.columns = df.columns.str.strip()

env = Environment()
template = env.from_string(TEMPLATE_STRING)
chapter_commands = defaultdict(list)
chapter_names = []

for chap, grp in df.groupby('章节'):
    chap_name = sanitize_filename(str(chap).strip())
    chap_dir = os.path.join(OUTPUT_DIR, chap_name)
    os.makedirs(chap_dir, exist_ok=True)
    chapter_names.append(chap_name)

    for _, row in grp.iterrows():
        cmd = str(row['命令']).strip()
        ctype = str(row.get('命令类型', '')).strip()
        out_file = os.path.join(chap_dir, sanitize_filename(f"{cmd}_{ctype}.rst"))

        # 响应字段（已在 expand 时处理）
        resp = str(row.get('响应', '')).strip()

        try:
            parameters = json.loads(row.get('参数json', '{}'))
        except Exception:
            parameters = {}

        if isinstance(parameters, list):
            pdict = {}
            for p in parameters:
                name = p.get('name', '').strip()
                desc = p.get('desc', '').strip()
                valmap = p.get('valmap', {}) or {}
                pdict[name] = {'__desc__': desc}
                for k, v in valmap.items():
                    pdict[name][k] = v
            parameters = pdict

        content = template.render(
            command_name=cmd,
            command_title=str(row.get('命令标题', '')).strip(),
            command_type=ctype,
            command=str(row.get('命令格式', '')).strip(),
            response=resp,
            description=str(row.get('功能描述', '')).strip(),
            note=str(row.get('备注', '')).strip(),
            parameters=parameters,
            example=str(row.get('示例命令', '')).strip()
        )

        with open(out_file, 'w', encoding='utf-8') as f:
            f.write(content)

        chapter_commands[chap_name].append(f"{cmd}_{ctype}")

print("✅ 所有 RST 文件已生成完毕")

✅ 所有 RST 文件已生成完毕


In [23]:
from jinja2 import Template

chapter_tmpl = """{{ chapter }}
{{ '=' * chapter|length }}

.. toctree::
   :maxdepth: 1

{% for c in cmds %}   {{ c }}
{% endfor %}
"""

main_tmpl = """AT 指令文档
===============

.. toctree::
   :maxdepth: 1
   :caption: 章节目录

{% for ch in chapter_names %}   {{ ch }}/index
{% endfor %}
"""

for chap, cmds in chapter_commands.items():
    idx = os.path.join(OUTPUT_DIR, chap, 'index.rst')
    with open(idx, 'w', encoding='utf-8') as f:
        f.write(Template(chapter_tmpl).render(chapter=chap, cmds=cmds))

main_idx = os.path.join(OUTPUT_DIR, 'index.rst')
with open(main_idx, 'w', encoding='utf-8') as f:
    f.write(Template(main_tmpl).render(chapter_names=chapter_names))

print("✅ 所有章节 index.rst 已生成")

✅ 所有章节 index.rst 已生成


In [24]:
!sphinx-build -b html docs/source docs/build/html -c docs/source
print("✅ HTML 构建完成，可查看 docs/build/html/index.html")

Running Sphinx v8.2.3
loading translations [en]... done
loading pickled environment... The configuration has changed (2 options: 'html_permalinks_icon', 'jquery_use_sri')
done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 5 source files that are out of date
updating environment: 0 added, 5 changed, 0 removed
reading sources... [ 20%] 2/AT+CSMS_执行
reading sources... [ 40%] 2/AT+CSMS_查询
reading sources... [ 60%] 2/AT+CSMS_测试
reading sources... [ 80%] 2/index
reading sources... [100%] index

looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
copying assets... 
copying static files... 
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\build\html\_static\basic.css
Writing evaluated template result to C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-le

C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_执行.rst:5: WARNING: duplicate label cmd-at+csms, other instance in C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_测试.rst
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_查询.rst:61: WARNING: Literal block expected; none found. [docutils]
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_查询.rst:5: WARNING: duplicate label cmd-at+csms, other instance in C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_执行.rst
C:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\learning-code\csv2pdf-02\docs\source\2\AT+CSMS_测试.rst:61: WARNING: Literal block expected; none found. [docutils]
C:\Use